In [16]:
from tensorflow.keras.models import Model
import tensorflow as tf
from data_loader import data_generator
from model import vit
import keras.backend as K
from helpers import f1
import warnings
import numpy as np
warnings.filterwarnings("ignore")

In [17]:
model = vit()

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[f1])
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_4 (InputLayer)        [(None, 256, 256, 3)]        0         []                            
                                                                                                  
 conv2d_12 (Conv2D)          (None, 128, 128, 64)         9472      ['input_4[0][0]']             
                                                                                                  
 conv2d_13 (Conv2D)          (None, 64, 64, 128)          73856     ['conv2d_12[0][0]']           
                                                                                                  
 conv2d_14 (Conv2D)          (None, 32, 32, 256)          295168    ['conv2d_13[0][0]']           
                                                                                            

In [18]:
train_generator = data_generator(
    image_dir='data_v3_processed/train/images/images',
    mask_dir='data_v3_processed/train/masks/masks',
    batch_size=64
)

val_generator = data_generator(
    image_dir='data_v3_processed/val/images/images',
    mask_dir='data_v3_processed/val/masks/masks',
    batch_size=64
)

# Get one batch to test
X_batch, y_batch = next(train_generator)
print(f"Image batch shape: {X_batch.shape}")  # Should be (4, 256, 256, 3)
print(f"Mask batch shape: {y_batch.shape}")   # Should be (4, 256, 256, 1)
print(f"Image range: [{X_batch.min():.3f}, {X_batch.max():.3f}]")
print(f"Mask range: [{y_batch.min():.3f}, {y_batch.max():.3f}]")

Image batch shape: (64, 256, 256, 3)
Mask batch shape: (64, 256, 256, 1)
Image range: [0.000, 0.937]
Mask range: [0.000, 1.000]


In [19]:
history = model.fit(train_generator,
                    validation_data=val_generator,
                    epochs=1,
                    batch_size=64,
                    steps_per_epoch=6000,
                    validation_steps=13)

6000/6000 [==============================] - 2061s 341ms/step - loss: 0.0042 - f1: 0.5564 - val_loss: 0.0119 - val_f1: 0.6900


In [20]:
model.save("vit_model.h5")